# Classification

## What is a classifier?

A classifier is some kind of rule / black box / widget that you can feed a new observation/data/record and it will decide whether or not it is part of a given class. E.g. below, we are classifying the animals to be either *cat* or *not cat*.

![A classifier for cats and not cats.](../fig/50-CatNotCat.jpg)

You can have classifiers for anything you can have a yes/no answer to, e.g.

- Is this a cat? 🐱
- Do these test results indicate cancer? 🚑
- Is this email spam or not spam? 📧

You can also have classifiers that categorise things into multiple (more than two) categories e.g.

- Which animal is this, out of the 12 animals I have trained my model on? 🐱
- Do these test results indicate {none, stage 1, stage 2, stage 3, stage 4} cancer? 🚑
- Is this email important, not important but not spam, or spam? 📧

It is clear that in some of these examples we are more concerned with being wrong in one direction than the other, e.g. it's better to let some spam email through accidentally than to block all of it but also junk important emails from people you know. Likewise, we would prefer our medical tests to err on the side of caution and not give a negative test result to someone who needs treatment. So we will need to adjust a parameter to decide how much we want to trade this off.

## Model evaluation (classification)

For now, let's imagine we have a classifier already. How can we test it to see how good it is?
A good start is a confusion matrix - a table of what test data it labels correctly and incorrectly.

![An demonstration of a confusion matrix for a cat classifier that has labelled 100 animals as cats or not-cats.](../fig/50-CatConfusion.jpg)

### Confusion Matrix

When applying classification models, we often use a confusion matrix to evaluate certain performance measures. A confusion matrix is simply a matrix that compares "the truth" to the labels generated by your classifier. When we label a cat correctly, we refer to this as a true positive. When we fail to label a cat as a cat, this is called a false negative.  However, if we label something which is not a cat as a cat, this is called a false positive; and of course if we correctly label something which is not a cat, as not a cat, then this is a true negative.

### Some common metrics

#### Accuracy: 
How often does the classifier label examples correctly? Objective: maximize. Example: 

$$\frac{TP+TN}{TP+TN+FP+FN} = \frac{\text{Correctly labelled examples}}{\text{All examples}}=\frac{31+52}{31+52+10+7}=83
\%$$

Accuracy is the opposite of the misclassification rate. So,

$$\text{Misclassification rate} = 1 - \text{Accuracy} = \frac{\text{Incorrectly labelled examples}}{\text{All examples}} $$

#### Precision: 
What fraction of things labelled as a cat were actually cats? Objective: maximize. Example:

$$\frac{TP}{TP+FP} = \frac{\text{Correctly labelled cats}}{\text{All things labelled as cats}}=\frac{31}{31+10}=76\%$$

#### Sensitivity / Recall: 
How often does the classifier label a cat as a cat? Objective: maximize. Example: 

$$\frac{TP}{TP+FN} = \frac{\text{Correctly labelled cats}}{\text{All true cats}}=\frac{31}{31+7}=81\%$$

#### Specificity: 
How often does it label a not-cat as a not-cat? Objective: maximize. Example: 

$$\frac{TN}{TN+FP} = \frac{\text{Correctly labelled not-cats}}{\text{All true not-cats}}=\frac{52}{52+10}=84\%$$

#### F1-score:

This is a commonly used overall measure of classifier performance (but not the only one and not always the best depending upon the problem). It is defined as the harmonic mean of precision and sensitivity;

$$\frac{1}{F_1} = \frac{1}{2}\left(\frac{1}{\text{Precision}}+\frac{1}{\text{Sensitivity}}\right) $$
So that
$$F_1 = 2\cdot\left(\frac{1}{\frac{1}{81\%}+\frac{1}{83\%}}\right) = 82\%$$

#### Mean-square error (MSE)

Define a loss function $L_i = 1$ if the $i$th example is classified incorrectly and $L_i = 0$ if it is classified correctly. If there are $N$ examples in total then the mean-square error is

$$\text{MSE} = \frac{1}{N}\sum_i L_i = \frac{\text{Incorrectly labelled examples}}{\text{All examples}}=\text{Misclassification Rate}$$

Which is actually just the misclassification rate above. 

#### AUC: Area under the curve

A good classifier will have high precision and high specificity, minimizing both false positives and false negatives. In practice, and with an imperfect classifier, you can tune a knob to say which of those two you care more about. There will be some kind of a trade-off between the two.

To capture this balance, we often use a Receiver Operator Characteristic (ROC) curve that plots the false positive rate along the x-axis and the true positive rate along the y-axis, for all possible trade-offs. A line that is diagonal from the lower left corner to the upper right corner represents a random guess at labelling each example. The higher the line is in the upper left-hand corner, the better the classifier in general. AUC computes the area under this curve. For a perfect classifier, AUC = 1, for a random guess, AUC=0.5. Objective: maximize.

![A Receiver Operator Characteristic (ROC) curve, from which the Area Under the Curve (AUC) can be calculated.](../fig/50-CatArea.jpg)


# Wisconsin Diagnostic Breast Cancer Dataset

Today we're going to be classifying patient's biopsy to see whether their breast tumor is malignant or benign. First, a fine needle aspirate (FNA) of a breast mass is taken. Basically they stick a needle in you and take a chunk out then put it under the microscope. It looks like this:

![An example of a Fine Needle Aspiration Biopsy](../fig/50-742_FNA1.jpg)

Features are computed from the digitized image, which describe the characteristics of the cell nuclei present in the image.

## Attribute Information:

1) ID number
2) Diagnosis (M = malignant, B = benign)
3-32)

Ten real-valued features are computed for each cell nucleus, each has 
- a *mean* across cells 
- a *standard deviation* across cells and 
- the *worst* value across cells:

a) radius (mean of distances from center to points on the perimeter)
b) texture (standard deviation of gray-scale values)
c) perimeter
d) area
e) smoothness (local variation in radius lengths)
f) compactness (perimeter^2 / area - 1.0)
g) concavity (severity of concave portions of the contour)
h) concave points (number of concave portions of the contour)
i) symmetry
j) fractal dimension ("coastline approximation" - 1)


## Let's Explore our data

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import model_selection
#from sklearn.cross_validation import train_test_split
%matplotlib inline
sns.set(font_scale = 1.5)

### Load Data

In [ ]:
wdbc_raw = pd.read_csv('../data/breast-cancer-wisconsin.csv') # read csv
wdbc = wdbc_raw.copy()

Fix the problem of whitespace in column names by replacing with underscores

In [ ]:
#print(wdbc.columns)
#wdbc.rename(columns=lambda x: x.strip())
wdbc.columns = wdbc.columns.str.replace(' ', '_', regex=True)
#print(wdbc.columns)

Recode diagnosis into integers

In [ ]:
#wdbc['diagnosis']
#wdbc['diagnosis'].astype("category").cat.codes

### Summary info
Shape of data frame

In [ ]:
wdbc.shape

Look for non-NAs

In [ ]:
wdbc.count()

In [ ]:
wdbc.describe()

### Explore relationship between variables
Plot mean radius of cell nucleii vs. mean concavity, coloured by diagnosis:

In [ ]:
_ = sns.lmplot('radius_mean','concavity_mean',data=wdbc, hue='diagnosis')

If we wanted to look at all possible scatterplot pairs we would do something like:

In [ ]:
_ = sns.pairplot(data=wdbc,
             vars=wdbc.columns[2:10],
             hue='diagnosis',
             markers=["o", "s"])

But it's easier to look at a correlation plot:

In [ ]:
catVars = wdbc.select_dtypes(include = ['object']).columns
numericVars = wdbc.select_dtypes(exclude = ['object']).columns

In [ ]:
corr = wdbc[numericVars[1:-1]].corr() # excluding id & Unnamed_32

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
_ = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0., square=True, linewidths=.5)

### Testing Distributions

In [ ]:
_ = sm.qqplot(
    wdbc['radius_mean'],
    line = 's'
)

In [ ]:
_ = sm.qqplot(
    np.sqrt(wdbc['radius_mean']),
    line = 's'
)

_ = sm.qqplot(
    np.log(wdbc['radius_mean']),
    line = 's'
)

## Aim

To create a classifier for predicting whether a breast cancer patient's tumor is malignant or benign.

#### Train Test Split

We're going to split our data into 70% training and 30% testing.

In [ ]:
rs = np.random.RandomState(33)

In [ ]:
# remove id, diagnosis, and Unnamed
predictors = wdbc.columns.values.tolist()
predictors.remove('id')
predictors.remove('diagnosis')
predictors.remove('Unnamed:_32')
print(predictors)

In [ ]:
features_train, features_test, outcome_train, outcome_test = model_selection.train_test_split(wdbc[predictors],wdbc['diagnosis'],test_size=0.3)

How many examples do we have in the training and testing sets?

In [ ]:
features_train.shape

In [ ]:
features_test.shape

## Let's Classify!

### k-Nearest Neighbours Classifier

This takes the nearest k things and and says what is the majority vote? E.g. in the example below we look at the seven nearest neighbours, 4 of which are cats so we say that the new example is probably a cat as well.

![A way to classify a new example as a cat or not...take the average of the nearest k=7 examples. It's a cat!](../fig/50-CatKNN.jpg)


Fit KNN classifier with a pre-determine number of neighbours

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=4)
knn = neigh.fit(features_train, outcome_train)

In [ ]:
outcome_pred_class = neigh.predict(features_test)
#outcome_pred_class.shape

In [ ]:
outcome_pred_prob = neigh.predict_proba(features_test)
outcome_pred_prob1 = [p[1] for p in outcome_pred_prob]

In [ ]:
#y_decision = knn.decision_function(features_test)

#### Classifier Diagnostics
So how well did the classifier do?

In [ ]:
y_true_outcome = outcome_test.astype("category").cat.codes.values
print(y_true_outcome)

In [ ]:
fpr, tpr, _ = roc_curve(y_true_outcome, outcome_pred_prob1)
AUC = roc_auc_score(y_true_outcome, outcome_pred_prob1)

plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = %4.2f'%AUC)
_ = plt.legend(loc="lower right")